In [ ]:
import os
GPU_NAME = os.popen('nvidia-smi --query-gpu=name --format=csv,noheader').read().strip()
os.environ['GPU_NAME'] = GPU_NAME
print(f'GPU: {GPU_NAME}')

GPU: Tesla T4


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!sudo apt-get update -y
!sudo apt-get install python3.8 -y
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1

In [ ]:
!sudo apt-get install python3.8-distutils
!sudo apt install python3.8-dev

In [ ]:
!python3 --version
!sudo apt install python3-pip

In [ ]:
import sys
from pathlib import Path

In [ ]:
%cd /content
!git clone --depth 1 https://github.com/sce-tts/TTS.git -b sce-tts
%cd /content/TTS

!pip install scipy==1.5 resampy pandas==1.3.4 matplotlib numpy==1.18.5
!pip install --upgrade pip
!pip uninstall numba -y
!pip uninstall resampy -y
!pip install resampy==0.3.1
!pip install numba==0.53.0
!pip install pydantic==1.10.10
!pip install -r requirements.txt
!pip install tensorboard
!pip install unidecode
!sudo apt-get remove python-blinker
!pip install --ignore-installed blinker
!python setup.py develop

%cd /content
!git clone --depth 1 https://github.com/sce-tts/TTS.git -b sce-tts
%cd /content/TTS

!pip install scipy==1.5 resampy pandas==1.3.4 matplotlib numpy==1.18.5
!pip install --upgrade pip
!pip uninstall numba -y
!pip uninstall resampy -y
!pip install resampy==0.3.1
!pip install numba==0.53.0
!pip install pydantic==1.10.10
!pip install -r requirements.txt
!pip install tensorboard
!pip install unidecode
!sudo apt-get remove python-blinker
!pip install --ignore-installed blinker
!python setup.py develop

In [ ]:
%cd /content/TTS
!cp "/content/drive/My Drive/Colab Notebooks/data/filelists.zip" ./filelists.zip
!rm -rf ./filelists
!unzip -q filelists.zip -d ./filelists

/content/TTS


In [ ]:
%cd /content/TTS
!mkdir -p "/content/drive/My Drive/Colab Notebooks/data/glowtts-v2"
if not Path("/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/config.json").exists():
    !gdown --id 1DMKLdfZ_gzc_z0qDod6_G8fEXj0zCHvC -O glowtts-v2.zip
    !unzip -q glowtts-v2.zip -d ./
    !cp -R ./glowtts-v2/* "/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/"

/content/TTS


In [ ]:
%cd /content/TTS
if not Path("/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/scale_stats_new.npy").exists():
    !python TTS/bin/compute_statistics.py "/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/config.json" "/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/scale_stats_new.npy" --data_path "/content/TTS/filelists/filelists/wavs/"

/content/TTS


In [ ]:
with open("/content/TTS/test_sentences.txt", mode="w") as f:
    f.write("""
테이블에 앉으셔도 되고 안쪽으로 들어가 편히 앉으셔도 돼요.
닭장 저쪽 구석에서 닭들의 천적인 수리부엉이 한 마리가 닭 한 마리를 낚아채 여유롭게 먹고 있다.
당신이 유럽에서 아시아로 삶의 터전을 옮기기에는 위험 부담이 있어요.
라마준은 얇은 밀가루 반죽 위에 고기를 얇게 펴서 구워낸 음식입니다.
나의 인생은 외곬길이었다.
부디 계약 내용을 찬찬히 훑어보시고 서명 후 저희에게 사본을 보내주십시오.
""")

In [ ]:
!pip install zipp
%reload_ext tensorboard
%tensorboard --logdir="/content/drive/My Drive/Colab Notebooks/data/glowtts-v2"

# **학습 진행**



In [ ]:
# 파일 경로 지정
file_path = '/content/TTS/TTS/bin/train_glow_tts.py'


# 파일 열기 및 읽기
with open(file_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()

# 파일 수정
modified_lines = []
for i, line in enumerate(lines):
    if 'test_sentences_file' in line:
        modified_lines.append(line.replace('with open(config.test_sentences_file, "r") as f:', 'with open(config.test_sentences_file, "r", encoding="utf-8") as f:', 1))
    elif 'test_sentences =' in line:
        modified_lines.append(line.replace('test_sentences = [s.strip() for s in f.readlines()]', 'test_sentences = [s.strip() for s in f.readlines()]', 1))
    else:
        modified_lines.append(line)

# 수정된 내용으로 파일 저장
with open(file_path, 'w', encoding='utf-8') as f:
    f.writelines(modified_lines)

In [ ]:
!(python TTS/bin/train_glow_tts.py \
    --config_path "/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/config.json" \
    --coqpit.datasets.0.path "/content/TTS/filelists/filelists"  \
    --coqpit.audio.stats_path "/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/scale_stats_new.npy"  \
    --coqpit.test_sentences_file "/content/TTS/test_sentences.txt"  \
    --coqpit.output_path "/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/"  \
    --coqpit.num_loader_workers 2  \
    --coqpit.num_val_loader_workers 2  \
    --restore_path "/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/model_file.pth.tar")

In [ ]:
%cd /content/TTS
!(python TTS/bin/train_glow_tts.py \
    --continue_path "/content/drive/MyDrive/Colab Notebooks/data/glowtts-v2/glowtts-v2-August-26-2024_01+16PM-3aa165a")

In [ ]:
import shutil
import os

# 삭제할 폴더 경로
folder_path = '/content/drive/MyDrive/Colab Notebooks/data/glowtts-v2'

# 폴더가 존재하는지 확인 후 삭제
if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
    print(f'{folder_path} has been deleted.')
else:
    print(f'{folder_path} does not exist.')